In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

In [2]:
csvfile = 'vehicle_stops_2016_datasd.csv'
df = pd.read_csv(csvfile)

In [3]:
df

,stop_id,stop_cause,service_area,subject_race,subject_sex,subject_age,timestamp,stop_date,stop_time,sd_resident,arrested,searched,obtained_consent,contraband_found,property_seized
0,1308198,Equipment Violation,530,W,M,28,2016-01-01 00:06:00,2016-01-01,0:06,Y,N,N,N,N,N
1,1308172,Moving Violation,520,B,M,25,2016-01-01 00:10:00,2016-01-01,0:10,N,N,N,NaN,NaN,NaN
2,1308171,Moving Violation,110,H,F,31,2016-01-01 00:14:00,2016-01-01,0:14,NaN,NaN,NaN,NaN,NaN,NaN
3,1308170,Moving Violation,Unknown,W,F,29,2016-01-01 00:16:00,2016-01-01,0:16,N,N,N,NaN,NaN,NaN
4,1308197,Moving Violation,230,W,M,52,2016-01-01 00:30:00,2016-01-01,0:30,N,N,N,NaN,NaN,NaN
5,1308200,Moving Violation,710,H,M,24,2016-01-01 00:30:00,2016-01-01,0:30,Y,N,N,NaN,NaN,NaN
6,1308174,Moving Violation,Unknown,O,M,20,2016-01-01 00:35:00,2016-01-01,0:35,Y,N,N,NaN,NaN,NaN
7,1308199,Moving Violation,440,H,M,50,2016-01-01 00:45:00,2016-01-01,0:45,Y,N,N,NaN,NaN,NaN
8,1308979,Moving Violation,310,H,F,25,2016-01-01 01:03:00,2016-01-01,1:03,Y,N,Y,N,N,N
9,1308965,Moving Violation,240,W,F,23,2016-01-01 01:10:00,2016-01-01,1:10,Y,N,N,NaN,NaN,NaN


In [4]:
mv_df = df[df.stop_cause == 'Moving Violation']
mv_df = mv_df[mv_df.service_area != 'Unknown']
mv_df = mv_df[mv_df.subject_age != 'No Age']
del mv_df['obtained_consent']
del mv_df['contraband_found']
del mv_df['property_seized']
mv_df.dropna(subset=['timestamp'])
mv_df.dropna(inplace=True)

In [5]:
mv_df

,stop_id,stop_cause,service_area,subject_race,subject_sex,subject_age,timestamp,stop_date,stop_time,sd_resident,arrested,searched
1,1308172,Moving Violation,520,B,M,25,2016-01-01 00:10:00,2016-01-01,0:10,N,N,N
4,1308197,Moving Violation,230,W,M,52,2016-01-01 00:30:00,2016-01-01,0:30,N,N,N
5,1308200,Moving Violation,710,H,M,24,2016-01-01 00:30:00,2016-01-01,0:30,Y,N,N
7,1308199,Moving Violation,440,H,M,50,2016-01-01 00:45:00,2016-01-01,0:45,Y,N,N
8,1308979,Moving Violation,310,H,F,25,2016-01-01 01:03:00,2016-01-01,1:03,Y,N,Y
9,1308965,Moving Violation,240,W,F,23,2016-01-01 01:10:00,2016-01-01,1:10,Y,N,N
10,1308175,Moving Violation,120,O,M,54,2016-01-01 01:20:00,2016-01-01,1:20,Y,N,N
11,1308176,Moving Violation,520,W,F,53,2016-01-01 01:39:00,2016-01-01,1:39,Y,N,N
12,1308177,Moving Violation,520,W,M,35,2016-01-01 01:57:00,2016-01-01,1:57,N,N,N
13,1308178,Moving Violation,520,W,M,29,2016-01-01 02:00:00,2016-01-01,2:00,N,Y,N
